In [3]:
#import libraries
%pip install face_recognition

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition


Note: you may need to restart the kernel to use updated packages.


In [4]:
#import libraries
import torch
from torch.autograd import Variable
import time
import os
import sys
import os
from torch import nn
from torchvision import models


In [9]:
#Model with feature visualization
from torch import nn
from torchvision import models
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [10]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax()
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))
def im_convert(tensor):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    cv2.imwrite('/content/drive/MyDrive/deep fake/ANIL.jpg',image*255)
    return image

def predict(model,img,path = './'):
  fmap,logits = model(img.to('cuda'))
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  logits = sm(logits)
  _,prediction = torch.max(logits,1)
  confidence = logits[:,int(prediction.item())].item()*100
  print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)
  idx = np.argmax(logits.detach().cpu().numpy())
  bz, nc, h, w = fmap.shape
  out = np.dot(fmap[-1].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  predict = out.reshape(h,w)
  predict = predict - np.min(predict)
  predict_img = predict / np.max(predict)
  predict_img = np.uint8(255*predict_img)
  out = cv2.resize(predict_img, (im_size,im_size))
  heatmap = cv2.applyColorMap(out, cv2.COLORMAP_JET)
  img = im_convert(img[:,-1,:,:,:])
  result = heatmap * 0.5 + img*0.8*255
  cv2.imwrite('/content/drive/MyDrive/deep fake/ANIL.jpg',result)
  result1 = heatmap * 0.5/255 + img*0.8
  r,g,b = cv2.split(result1)
  result1 = cv2.merge((r,g,b))
  plt.imshow(result1)
  plt.show()
  return [int(prediction.item()),confidence]
#img = train_data[100][0].unsqueeze(0)
#predict(model,img)

In [11]:
#!pip3 install face_recognition
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
class validation_dataset(Dataset):
    def __init__(self, video_names, sequence_length=60, transform=None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length

    def __len__(self):
        return len(self.video_names)

    def __getitem__(self, idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100 / self.count)
        first_frame = np.random.randint(0, a)
        for i, frame in enumerate(self.frame_extract(video_path)):
            faces = face_recognition.face_locations(frame)
            if faces:  # Proceed only if faces are detected
                top, right, bottom, left = faces[0]
                frame = frame[top:bottom, left:right, :]
                frames.append(self.transform(frame))
                if len(frames) == self.count:
                    break
        # Pad frames if there are not enough
        while len(frames) < self.count:
            frames.append(torch.zeros_like(frames[0]))
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)

    def frame_extract(self, path):
        vidObj = cv2.VideoCapture(path)
        success = 1
        while success:
            success, image = vidObj.read()
            if success:
                yield image


In [19]:
import os
path_to_model = r"D:\DEEP FAKE COLOB\model_87_acc_20_frames_final_data.pt"
print(os.path.exists(path_to_model))


True


In [25]:
import torch
from torchvision import transforms
# Import your Model class, validation_dataset, etc.
# from model_file import Model, validation_dataset

# ----------------------------
# Device setup (GPU/CPU)
# ----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ----------------------------
# Video preprocessing
# ----------------------------
im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# ----------------------------
# Video paths
# ----------------------------
path_to_videos = [r"D:\DEEP FAKE COLOB\FAKE 2.mp4"]

# Load dataset
video_dataset = validation_dataset(
    path_to_videos, sequence_length=20, transform=train_transforms
)

# ----------------------------
# Load model
# ----------------------------
model = Model(2)  # Initialize model
path_to_model = r"D:\DEEP FAKE COLOB\model_87_acc_20_frames_final_data.pt"

# Load state dict safely on CPU
state_dict = torch.load(path_to_model, map_location='cpu')
model.load_state_dict(state_dict)
model.eval()
model.to(device)  # Move model to the correct device

# ----------------------------
# Predict function
# ----------------------------
def predict(model, img, path='./', device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = model.to(device)
    img = img.to(device)

    with torch.no_grad():
        fmap, logits = model(img)
    
    prediction = logits.detach().cpu().numpy()
    
    # Handle batch dimension
    if prediction.ndim > 1:
        prediction = prediction[0]

    return prediction


# ----------------------------
# Prediction loop
# ----------------------------
for i, video_path in enumerate(path_to_videos):
    print(f"Processing: {video_path}")
    
    video_tensor = video_dataset[i]
    
    # Make prediction
    prediction = predict(model, video_tensor, './', device=device)
    
    # Convert to predicted class
    pred_class = prediction.argmax()
    
    if pred_class == 1:
        print("REAL")
    else:
        print("FAKE")


Using device: cpu


d:\anaconda\envs\deepfake_conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda\envs\deepfake_conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing: D:\DEEP FAKE COLOB\FAKE 2.mp4
FAKE


# Deepfake Model Accuracy Checker

In [26]:
import os
import torch
from torchvision import transforms
# Import your Model, validation_dataset, predict functions
# from model_file import Model, validation_dataset, predict

# ----------------------------
# Device setup
# ----------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ----------------------------
# Load model
# ----------------------------
model = Model(2)
path_to_model = r"D:\DEEP FAKE COLOB\model_87_acc_20_frames_final_data.pt"
state_dict = torch.load(path_to_model, map_location='cpu')  # load safely
model.load_state_dict(state_dict)
model.eval()
model.to(device)

# ----------------------------
# Video transforms
# ----------------------------
im_size = 112
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((im_size, im_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# ----------------------------
# Accuracy function
# ----------------------------
def calculate_accuracy(test_folder):
    total = 0
    correct = 0
    
    for label_str in ['real', 'fake']:  # folder names
        label = 1 if label_str == 'real' else 0
        folder_path = os.path.join(test_folder, label_str)
        videos = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.mp4')]
        
        dataset = validation_dataset(videos, sequence_length=20, transform=train_transforms)
        
        for i, video_tensor in enumerate(dataset):
            prediction = predict(model, video_tensor, device=device)
            # Ensure 1D array
            if prediction.ndim > 1:
                prediction = prediction[0]
            pred_class = prediction.argmax()
            
            video_name = os.path.basename(videos[i])
            pred_label = "REAL" if pred_class == 1 else "FAKE"
            print(f"{video_name}: Predicted = {pred_label}, True = {label_str.upper()}")
            
            if pred_class == label:
                correct += 1
            total += 1
    
    accuracy = correct / total if total > 0 else 0
    print(f"\nOverall Accuracy: {accuracy*100:.2f}%")
    return accuracy

# ----------------------------
# Run accuracy check
# ----------------------------
test_folder = r"D:\DEEP FAKE COLOB\test_videos"
calculate_accuracy(test_folder)


Using device: cpu
REAL 1.mp4: Predicted = FAKE, True = REAL
REAL 10.mp4: Predicted = REAL, True = REAL
REAL 2.mp4: Predicted = REAL, True = REAL
REAL 3.mp4: Predicted = REAL, True = REAL
REAL 4.mp4: Predicted = REAL, True = REAL
REAL 5.mp4: Predicted = FAKE, True = REAL
FAKE 1.mp4: Predicted = FAKE, True = FAKE
FAKE 10.mp4: Predicted = FAKE, True = FAKE
FAKE 2.mp4: Predicted = FAKE, True = FAKE
FAKE 3.mp4: Predicted = REAL, True = FAKE
FAKE 4.mp4: Predicted = FAKE, True = FAKE
FAKE 5.mp4: Predicted = FAKE, True = FAKE
FAKE 6.mp4: Predicted = REAL, True = FAKE
FAKE 7.mp4: Predicted = FAKE, True = FAKE
FAKE 8.mp4: Predicted = FAKE, True = FAKE
FAKE 9.mp4: Predicted = REAL, True = FAKE

Overall Accuracy: 68.75%


0.6875